# FHIR-Drills (Simple Patient)

This notebook is a rough, Jupyter translation of the FHIR-Drills tutorial based on Postman avaialble [here](https://fhir-drills.github.io/simple-patient.html). Rather than using the Python Smart on FHIR library, I'm going to use the Python requests library so that we are working more directly with the HTTP protocol and are closer to the steps described in the original tutorial than would be achieved with the FHIR Python client.

For more details, please refer to the original tutorials.

FHIR is built upon web standards, particularly [Representational state transfer (REST)](https://en.wikipedia.org/wiki/Representational_state_transfer). 

In this tutorial we will explore RESTful operations to create, read, update, and delete resources (CRUD) with a FHIR server. 

We will interact with the FHIR server using the Hype Text Transfer Protocol [(HTTP)](https://en.wikipedia.org/wiki/Hypertext_Transfer_Protocol), a digital communications protocol develooped by Tim Berners-Lee at CERN. We will use the Python package [`requests`](https://requests.readthedocs.io/en/master/index.html) to implement the HTTP commands for us. `requests` will generate URLs that tell the remote server what we want to do, receive the responses from the remote servers, and convert them into Python objects we can interact with.

## Find a patient with relevant variables

In [1]:
import requests
from post_data import create_resource
import pprint
pp = pprint.PrettyPrinter(indent=2)

### Create Patient and get `id`

I've written some Python code to upload a patient to the server behidn the scences, mimicing what the HTML button does on the original tutorial

When we invoke `create_resource()` we upload a JSON file describing a patient. When this is posted (inserted) into the remote server, this patient is assigned a unique identifier which is necessary to interact with the patient data remotely. This identifer is then stored in the variable `patientId`.

In [2]:
patientId = create_resource()

In [3]:
patientId

'afbf6c1f-ff4c-452a-9cab-8812ff486821'

### Set the headers to work with FHIR data

Data transfer with HTTP relies on specifying the [media type (MIME type)](https://en.wikipedia.org/wiki/Hypertext_Transfer_Protocol). This is done with the HTTP header. So we need to specify that we will be using FHIR data. We will also specify what character encoding we will be using.

Finally, we need to know the Uniform Resource Locator (URL) of the FHIR server.

In [4]:
FHIRJSONMimeType = 'application/fhir+json'
header_defaults = {
            'Accept': FHIRJSONMimeType,
            'Accept-Charset': 'UTF-8',
        }
URL = 'https://stu3.test.pyrohealth.net/fhir'

To request the FHIR resource, we build up a URL based on

- The base URL: `URL`
- The resource type we want: e.g. `Patient`
- The ID of the resoruce we want: e.g. `patientID` (returned from `create_resource()`)

In [5]:
rurl = URL+ '/Patient/'+patientId
rurl

'https://stu3.test.pyrohealth.net/fhir/Patient/afbf6c1f-ff4c-452a-9cab-8812ff486821'

## Get Our Patient

We will use the HTTP [GET](https://www.w3.org/Protocols/rfc2616/rfc2616-sec9.html) method.

In [6]:
r = requests.get(rurl,
                 headers = header_defaults)

`r` contains the results of our request. It has multiple parts

- `r.status_code`: Is the code returned by the remote server. `200` means success! I ran across many bad codes trying to get things working.
- `r.headers`: The metadata about the request and response
- `r.text`:
- `r.json`: A method to convert the data returned by the server into a Python dictionary.

In [7]:
print(r.status_code)
print(r.headers)

200
{'Content-Type': 'application/fhir+json; charset=utf-8', 'Content-Encoding': 'gzip', 'Last-Modified': 'Wed, 14 Jul 2021 08:58:49 GMT', 'ETag': 'W/"1"', 'Location': 'https://stu3.test.pyrohealth.net/fhir/Patient/afbf6c1f-ff4c-452a-9cab-8812ff486821', 'Server': 'Microsoft-IIS/8.5', 'X-Powered-By': 'ASP.NET', 'Date': 'Wed, 14 Jul 2021 09:02:02 GMT', 'Content-Length': '658'}


In [8]:
print(r.text)

{"resourceType":"Patient","id":"afbf6c1f-ff4c-452a-9cab-8812ff486821","meta":{"versionId":"1","lastUpdated":"2021-07-14T08:58:49.516+00:00"},"text":{"status":"generated","div":"<div xmlns=\"http://www.w3.org/1999/xhtml\"><p>Patient: Fhirman, Sam</p></div>"},"identifier":[{"type":{"coding":[{"system":"http://hl7.org/fhir/v2/0203","code":"NI","display":"National unique individual identifier"}],"text":"IHI"},"system":"http://ns.electronichealth.net.au/id/hi/ihi/1.0","value":"8003608166690503"},{"use":"usual","type":{"coding":[{"system":"http://hl7.org/fhir/v2/0203","code":"MR"}]},"system":"urn:oid:1.2.36.146.595.217.0.1","value":"6666","period":{"start":"2001-05-06"},"assigner":{"display":"Acme Healthcare"}}],"name":[{"use":"official","text":"Sam Fhirman","family":"Fhirman","given":["Sam"],"prefix":["Mr"]}],"telecom":[{"system":"phone","value":"+61481059995","use":"mobile"}],"gender":"male","birthDate":"1973-09-30","address":[{"use":"work","line":["400 George Street"],"city":"Brisbane","s

In [9]:
pp.pprint(r.json())

{ 'address': [ { 'city': 'Brisbane',
                 'country': 'AUS',
                 'line': ['400 George Street'],
                 'postalCode': '4000',
                 'state': 'QLD',
                 'use': 'work'}],
  'birthDate': '1973-09-30',
  'gender': 'male',
  'id': 'afbf6c1f-ff4c-452a-9cab-8812ff486821',
  'identifier': [ { 'system': 'http://ns.electronichealth.net.au/id/hi/ihi/1.0',
                    'type': { 'coding': [ { 'code': 'NI',
                                            'display': 'National unique '
                                                       'individual identifier',
                                            'system': 'http://hl7.org/fhir/v2/0203'}],
                              'text': 'IHI'},
                    'value': '8003608166690503'},
                  { 'assigner': {'display': 'Acme Healthcare'},
                    'period': {'start': '2001-05-06'},
                    'system': 'urn:oid:1.2.36.146.595.217.0.1',
                  

## Get the returned content

We are going to extract the returned data and use the resposne objects `json` method to convert the returned object to a Python dictionary.

In [10]:
patient = r.json()

In [11]:
pp.pprint(patient)

{ 'address': [ { 'city': 'Brisbane',
                 'country': 'AUS',
                 'line': ['400 George Street'],
                 'postalCode': '4000',
                 'state': 'QLD',
                 'use': 'work'}],
  'birthDate': '1973-09-30',
  'gender': 'male',
  'id': 'afbf6c1f-ff4c-452a-9cab-8812ff486821',
  'identifier': [ { 'system': 'http://ns.electronichealth.net.au/id/hi/ihi/1.0',
                    'type': { 'coding': [ { 'code': 'NI',
                                            'display': 'National unique '
                                                       'individual identifier',
                                            'system': 'http://hl7.org/fhir/v2/0203'}],
                              'text': 'IHI'},
                    'value': '8003608166690503'},
                  { 'assigner': {'display': 'Acme Healthcare'},
                    'period': {'start': '2001-05-06'},
                    'system': 'urn:oid:1.2.36.146.595.217.0.1',
                  

Dictionary are __key__/__value__ pairs.

We can list the keys with the `keys()` method.

In [12]:
patient.keys()

dict_keys(['resourceType', 'id', 'meta', 'text', 'identifier', 'name', 'telecom', 'gender', 'birthDate', 'address', 'maritalStatus'])

We access the values in the dictionary using square brackets and the keys.

For example,

In [13]:
patient["birthDate"]

'1973-09-30'

We can change the value by assigning a value to the key

In [14]:
patient["birthDate"] = "2019-11-23"

## Change Patient name

The tutorial suggests we change the patient name to your own and then push the results up to the server.

#### To Do: Modify the values below to match your name rather than mine

In [15]:
patient["name"] = [{'family': "Chapman", 'given':['Brian', 'Earl'], 
              "prefix":"Dr.", "text":"Brian Chapman", "use":"official"}]

## PUT the modified data on the server

we will use the `PUT` method to push our modified patient to the server.

In [16]:
r2 = requests.put(URL+'/Patient/'+patientId,
                 headers = header_defaults,
                 json=patient)
r2.ok

True

`r2.ok` is a boolean value that basically tells you whether things went as planned.

## Verify Results

In [17]:
requests.get(URL+'/Patient/'+patientId,
                 headers = header_defaults).json()['name']

[{'use': 'official',
  'text': 'Brian Chapman',
  'family': 'Chapman',
  'given': ['Brian', 'Earl'],
  'prefix': ['Dr.']}]

### Data is Modified!

At least for me!

## Delete the Resource

To delete we simply use the delete method and provide the URL of the resource we want to delete.

In [18]:
r3 = requests.delete(URL+'/Patient/'+patientId)


In [19]:
print(r3.status_code)
print(r3.ok)

204
True


## Verify the patient is deleted.

In [20]:
rd = requests.get(URL+'/Patient/'+patientId,
                 headers = header_defaults)

In [21]:
rd.status_code

410

status code `410`  means `Gone`

In [23]:
versionid = 1
while True:
    print("VERSION: ", versionid)
    rtmp = requests.get(URL+'/Patient/'+patientId+"/_history/%d"%versionid,
                 headers = header_defaults)
    print(rtmp.status_code)
    try:
        pp.pprint(rtmp.json())
        print("-"*42)
        versionid += 1
    except:
        break
    

VERSION:  1


200
{ 'address': [ { 'city': 'Brisbane',
                 'country': 'AUS',
                 'line': ['400 George Street'],
                 'postalCode': '4000',
                 'state': 'QLD',
                 'use': 'work'}],
  'birthDate': '1973-09-30',
  'gender': 'male',
  'id': 'afbf6c1f-ff4c-452a-9cab-8812ff486821',
  'identifier': [ { 'system': 'http://ns.electronichealth.net.au/id/hi/ihi/1.0',
                    'type': { 'coding': [ { 'code': 'NI',
                                            'display': 'National unique '
                                                       'individual identifier',
                                            'system': 'http://hl7.org/fhir/v2/0203'}],
                              'text': 'IHI'},
                    'value': '8003608166690503'},
                  { 'assigner': {'display': 'Acme Healthcare'},
                    'period': {'start': '2001-05-06'},
                    'system': 'urn:oid:1.2.36.146.595.217.0.1',
              

200
{ 'address': [ { 'city': 'Brisbane',
                 'country': 'AUS',
                 'line': ['400 George Street'],
                 'postalCode': '4000',
                 'state': 'QLD',
                 'use': 'work'}],
  'birthDate': '2019-11-23',
  'gender': 'male',
  'id': 'afbf6c1f-ff4c-452a-9cab-8812ff486821',
  'identifier': [ { 'system': 'http://ns.electronichealth.net.au/id/hi/ihi/1.0',
                    'type': { 'coding': [ { 'code': 'NI',
                                            'display': 'National unique '
                                                       'individual identifier',
                                            'system': 'http://hl7.org/fhir/v2/0203'}],
                              'text': 'IHI'},
                    'value': '8003608166690503'},
                  { 'assigner': {'display': 'Acme Healthcare'},
                    'period': {'start': '2001-05-06'},
                    'system': 'urn:oid:1.2.36.146.595.217.0.1',
              

200
